In [3]:
import numpy as np

In [53]:

u = 4500
v = 8000
P_LEFTTOP = np.array([[2.95129075e+03, -2.82185939e+03, 1.57759623e+02, 1.41552395e+05],
              [-3.97081557e-01, 3.99639466e+01, 4.00827873e+03, 3.72912795e+04],
              [-4.99372034e-01, -8.54472868e-01, 1.43191094e-01, 4.48297288e+01]])

P_CENTER = np.array([[4.44365090e+03, -3.90197381e+02, 2.21990980e+02, 3.62449157e+04],  # Example camera projection matrix
              [-6.11777948e+01, 4.61190201e+02, 4.37048375e+03, 3.81440740e+04],
              [1.19823975e-01, -9.66444490e-01, 2.27216334e-01, 8.46945190e+01]])

P_RIGHTTOP = np.array ([[ 2.29866174e+03, 8.69940269e+02, 1.68312186e+02,-5.53368068e+04],
             [-1.51913690e+02, 2.03340472e+02, 2.39260229e+03, 3.14051325e+04],
             [ 5.58782189e-01, -7.89760745e-01, 2.53062111e-01, 5.33783778e+01]])

# Homogeneous 2D point in screen coordinates
image_coords = np.array([u + 105/2, v + 68/2, 1])  # Add offsets to match reverse process

# Reduced 3x3 projection matrix P'
P_reduced = P_CENTER[:, [0, 1, 3]]  # Remove the third column (Z terms)

# Compute inverse of P'
P_reduced_inv = np.linalg.inv(P_reduced)

# Compute 3D world coordinates (Z = 0 plane)
world_coords_homogeneous = P_reduced_inv @ image_coords
X, Y = world_coords_homogeneous[:2]

print(f"Real-world coordinates: X = {X}, Y = {Y}")

In [53]:
def pixel_to_pitch_location(pixel_coords, projection_matrix):
    """
    Converts pixel coordinates to real-world pitch coordinates assuming z=0.

    Parameters:
    - pixel_coords: tuple (x, y), the pixel coordinates in the image.
    - projection_matrix: numpy array (3x4), the camera projection matrix.

    Returns:
    - Tuple (x_world, y_world) representing the real-world coordinates on the pitch.
    """
    # Homogeneous representation of the 2D pixel coordinates (x, y)

    pixel_homogeneous = np.array([x, y, 1])

    # Extracting the homography matrix H (3x3) for the plane z=0 from P
    P = projection_matrix[:, :3]  # First three columns of P
    H_inv = np.linalg.inv(P)      # Inverse of the homography matrix

    # Project pixel to world coordinates on plane z=0
    world_coords_homogeneous = H_inv @ pixel_homogeneous
    world_coords_homogeneous /= world_coords_homogeneous[2]  # Normalize to make z=1

    x_world, y_world = world_coords_homogeneous[0], world_coords_homogeneous[1]
    return x_world, y_world

In [53]:
pixel_coords = u, v
pixel_to_pitch_location(pixel_coords, P)

In [53]:
# Define the matrix
P_LEFTTOP = np.array([[2.95129075e+03, -2.82185939e+03, 1.57759623e+02, 1.41552395e+05],
                   [-3.97081557e-01, 3.99639466e+01, 4.00827873e+03, 3.72912795e+04],
                   [-4.99372034e-01, -8.54472868e-01, 1.43191094e-01, 4.48297288e+01]])

P_CENTER = np.array([[4.44365090e+03, -3.90197381e+02, 2.21990980e+02, 3.62449157e+04],  # Example camera projection matrix
              [-6.11777948e+01, 4.61190201e+02, 4.37048375e+03, 3.81440740e+04],
              [1.19823975e-01, -9.66444490e-01, 2.27216334e-01, 8.46945190e+01]])

pixel_coords = np.array([822, 356, 1.0])
# Compute the pseudo-inverse


In [53]:
P_plus = np.linalg.pinv(P_LEFTTOP)

# Compute the 3D world coordinates
world_coords = P_plus @ pixel_coords

# Since Z = 0 in the world plane, we discard the Z-coordinate
world_coords_2d = world_coords[:2]  # Only X and Y

# Real-world plane dimensions (meters)
plane_width_meters = 105
plane_height_meters = 65

# Image dimensions (pixels)
image_width_pixels = 1920
image_height_pixels = 1080

# Scaling factors from pixels to meters
scale_x = plane_width_meters / image_width_pixels
scale_y = plane_height_meters / image_height_pixels

# Convert world coordinates from pixels to meters
x_real_world = world_coords_2d[0] * scale_x
y_real_world = world_coords_2d[1] * scale_y

print(f"Real-world coordinates (in meters): X = {x_real_world}, Y = {y_real_world}")